In [ ]:
# @title Cell 1 - Install & Setup (Manual Overdrive Version)
!pip install -U yt-dlp
import yt_dlp
import os
import re
import json
import time
import subprocess  # Added for manual FFmpeg control
from datetime import datetime

# --- CONFIG ---
OUTPUT_BASE = "/content/tiktok"
LOG_FILE = os.path.join(OUTPUT_BASE, "tiktok_downloader_log.txt")
FAILED_URLS_FILE = os.path.join(OUTPUT_BASE, "failed_urls.txt")
cookies_path = os.path.join(OUTPUT_BASE, "cookies.txt")

os.makedirs(OUTPUT_BASE, exist_ok=True)

def get_ydl_opts(use_cookies=True, extra_opts=None):
    opts = {
        'user_agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
        'http_headers': {
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'Referer': 'https://www.tiktok.com/',
            'Connection': 'keep-alive',
        },
        'sleep_interval': 2,
        'max_sleep_interval': 5,
        'no_warnings': True,
        'quiet': True,
    }
    if use_cookies and os.path.exists(cookies_path):
        opts['cookiefile'] = cookies_path
    if extra_opts:
        opts.update(extra_opts)
    return opts

def sanitize_filename(name):
    return re.sub(r'[<>:\"/\\\\|?*]', '_', name)

def generate_filename(info):
    return f"{info['username']}-{info['video_id']}-{info['desc']}.mp4"

def get_available_path(folder, filename):
    full = os.path.join(folder, filename)
    if not os.path.exists(full):
        return full
    i = 1
    while True:
        name, ext = os.path.splitext(filename)
        new = os.path.join(folder, f"{name}-{i}{ext}")
        if not os.path.exists(new):
            return new
        i += 1

def is_already_downloaded(info):
    user_folder = os.path.join(OUTPUT_BASE, info['username'])
    if not os.path.isdir(user_folder):
        return False
    prefix = f"{info['username']}-{info['video_id']}"
    return any(f.startswith(prefix) and f.endswith(".mp4") for f in os.listdir(user_folder))

def get_video_info(url, use_cookies=True):
    ydl_opts = get_ydl_opts(use_cookies=use_cookies)
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
    title = info.get('title', 'untitled').strip()
    return {
        'username': info.get('uploader', 'unknown'),
        'video_id': info.get('id', 'unknown'),
        'title': title,
        'desc': sanitize_filename(title[:60].lower().replace(' ', '-')),
        'url': url
    }

def log_download(info, status="Success", error=""):
    entry = {
        "timestamp": datetime.now().isoformat(),
        "url": info.get('url'),
        "username": info.get('username'),
        "video_id": info.get('video_id'),
        "title": info.get('title'),
        "status": status,
        "error": str(error) if error else ""
    }
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")
    if "Failed" in status:
        with open(FAILED_URLS_FILE, "a", encoding="utf-8") as f:
            f.write(info.get('url', '') + "\n")

def download_single(info, use_cookies=True):
    folder = os.path.join(OUTPUT_BASE, info['username'])
    os.makedirs(folder, exist_ok=True)

    final_filename = generate_filename(info)
    final_path = get_available_path(folder, final_filename)

    # 1. We download to a temporary .hvc1 file first
    temp_path = final_path + ".hvc1"

    ydl_opts = get_ydl_opts(
        use_cookies=use_cookies,
        extra_opts={
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': temp_path,
            'merge_output_format': 'mp4',
            'quiet': True,
        }
    )

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([info['url']])

    # 2. Manual FFmpeg transcode (The "Un-skippable" part)
    print(f"⚙️  Forcing transcode to AVC1 (H.264)...")
    ffmpeg_cmd = [
        'ffmpeg', '-y',        # Overwrite if exists
        '-i', temp_path,       # Input (HVC1)
        '-c:v', 'libx264',     # Force AVC1 codec
        '-pix_fmt', 'yuv420p', # Compatibility
        '-crf', '22',          # Quality
        '-preset', 'faster',     # Speed
        '-c:a', 'aac',          # Standard audio
        '-b:a', '128k',         # Standard audio bitrate
        final_path             # Output (AVC1)
    ]

    result = subprocess.run(ffmpeg_cmd, capture_output=True, text=True)

    if result.returncode == 0:
        # 3. Clean up the temporary file
        os.remove(temp_path)
        log_download(info, "Success")
    else:
        print(f"❌ FFmpeg Error: {result.stderr}")
        log_download(info, "Failed", result.stderr)

In [9]:
# link json
urls = [
    "",
    "",
    "",
    "",
    "",
    "",
    "",
]


In [ ]:
# @title Cell 2 - upload link txt
from google.colab import files

uploaded = files.upload()


In [ ]:
# @title Cell 2 - load url
txt_file = next(iter(uploaded))
with open(txt_file, "r", encoding="utf-8") as f:
    urls = [line.strip() for line in f if line.strip()]

print(f"✅ Loaded {len(urls)} URLs from file")


In [ ]:
# @title Cell 3 - download from tiktok
use_cookies = False  # set False if you don't upload cookies.txt

total = len(urls)
downloaded = 0
skipped = 0

for idx, url in enumerate(urls, 1):
    print(f"\n[{idx}/{total}] Processing:", url)

    try:
        info = get_video_info(url, use_cookies=use_cookies)

        if is_already_downloaded(info):
            print("⏭️ Skipped (already exists)")
            log_download(info, "Skipped")
            skipped += 1
            continue

        print(f"📥 Downloading @{info['username']}")
        download_single(info, use_cookies=use_cookies)
        downloaded += 1
        time.sleep(1)

    except Exception as e:
        print("❌ Failed:", e)
        log_download({"url": url}, "Failed", e)

print(f"\n✅ DONE — {downloaded} downloaded, {skipped} skipped")


In [ ]:
# @title Cell 4 - Download
import shutil

ZIP_NAME = "/content/tiktok_all_downloads.zip"

# remove old zip if exists
if os.path.exists(ZIP_NAME):
    os.remove(ZIP_NAME)

shutil.make_archive(
    base_name=ZIP_NAME.replace(".zip", ""),
    format="zip",
    root_dir=OUTPUT_BASE
)

print("✅ ZIP created:", ZIP_NAME)

from google.colab import files

files.download(ZIP_NAME)
